In [7]:
import numpy as np


def Skew(w):
    skew_mat = np.array([[0, -w[2], w[1]],
                                [w[2], 0, -w[0]],
                                [-w[1], w[0], 0]], dtype = object)
    return skew_mat

def matrix_exp(w_hat, theta):
    skew_w = Skew(w_hat)
    R = np.eye(3) + np.sin(theta) * skew_w + (1 - np.cos(theta)) * np.matmul(skew_w, skew_w)
    return R

def screw_exp(screw, theta):
    w_hat = screw[:3]
    v = screw[3:]

    w_hat_skew = Skew(w_hat)

    R = matrix_exp(w_hat, theta)
    G_theta = np.eye(3) * theta + (1 - np.cos(theta)) * w_hat_skew + (theta - np.sin(theta)) * np.matmul(w_hat_skew, w_hat_skew)

    T = np.vstack((np.hstack((R, np.matmul(G_theta, v.reshape((-1, 1))))),
                       [0, 0, 0, 1]))
    return T
    
def Ad(T):
    p = T[:3, 3]
    R = T[:3, :3]
    p_mat = Skew(p)
    Adjoint = np.vstack((np.hstack((R, np.zeros((3, 3)))),
                             np.hstack((p_mat @ R, R))))
    return Adjoint

def BodyTwist(V_bmat):
    w_b = np.array([V_bmat[2, 1], V_bmat[0, 2], V_bmat[1, 0]])
    w_b = w_b.reshape(-1,1)
    v_b = V_bmat[:3, 3]
    v_b = v_b.reshape(-1,1)
    output = np.vstack((w_b, v_b))
    return output

In [ ]:
import numpy as np
import time

from scipy.linalg import logm

q = np.array([0, 0.166, -0.01])

M = np.vstack([np.hstack([np.eye(3), q.reshape(-1, 1)]),
                  np.array([0, 0, 0, 1])])

w = np.array([[0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1]])
q = np.array([[0, 0, 0], [0, 0, 0.073], [0, 0.083, 0.073], [0, 0.166, 0.073], [0, 0.166, 0.033]])
v = -np.cross(w,q)

s1 = np.hstack([w[0], v[0]])
s2 = np.hstack([w[1], v[1]])
s3 = np.hstack([w[2], v[2]])
s4 = np.hstack([w[3], v[3]])
s5 = np.hstack([w[4], v[4]])
s = np.hstack([s1.reshape(-1,1), s2.reshape(-1,1), s3.reshape(-1,1), s4.reshape(-1,1), s5.reshape(-1,1)])

x_d = np.array([0.02, 0.12, 0.05]) 
x_d = x_d.reshape(-1,1)
T_sd = np.vstack([np.hstack([np.eye(3), x_d]),
                  np.array([[0, 0, 0, 1]])])
th = np.deg2rad([-86.9474, 41.3064, -44.3338, 3.0274, 86.9474])
th = th.reshape(-1,1)

dt = 1

exp1 = screw_exp(s[:,0].reshape(-1,1), th[0])
exp2 = screw_exp(s[:,1].reshape(-1,1), th[1])
exp3 = screw_exp(s[:,2].reshape(-1,1), th[2])
exp4 = screw_exp(s[:,3].reshape(-1,1), th[3])
exp5 = screw_exp(s[:,4].reshape(-1,1), th[4])
T_sb = exp1 @ exp2 @ exp3 @ exp4 @ exp5 @ M 
T_sb = T_sb.astype(dtype = 'float64')

T_bs = np.linalg.inv(T_sb)
AdT_bs = Ad(T_bs)

Js1 = s[:,0].reshape(-1,1)
Js2 = Ad(exp1) @ s[:,1].reshape(-1,1)
Js3 = Ad(exp1 @ exp2) @ s[:,2].reshape(-1,1)
Js4 = Ad(exp1 @ exp2 @ exp3) @ s[:,3].reshape(-1,1)
Js5 = Ad(exp1 @ exp2 @ exp3 @ exp4) @ s[:,4].reshape(-1,1)
Js = np.hstack([Js1, Js2, Js3, Js4, Js5])
Js = Js.astype(dtype = 'float64')
Jb = AdT_bs @ Js
Jb = Jb.astype(dtype = 'float64')

T_bd = T_bs @ T_sd
V_bmat = logm(T_bd) * 0.07
w_b = np.array([V_bmat[2, 1], V_bmat[0, 2], V_bmat[1, 0]])
v_b = V_bmat[:3, 3]
V_b = np.hstack([w_b, v_b])

th_center = np.zeros((1, 5))
th_center[0,2] = -np.pi / 3
th_center[0,3] = np.pi / 6
grad_H = np.zeros((1, 5))
grad_H[0,2] = th[2] - th_center[0,2]
grad_H[0,3] = th[3] - th_center[0,3]
grad_H = grad_H.reshape(-1,1)
gamma = -0.2

Jvb = np.linalg.pinv(Jb) @ V_b
Jvb = Jvb.reshape(-1,1)
dth = Jvb + gamma * (np.eye(5) - np.linalg.pinv(Jb) @ Jb) @ grad_H

th = th + dth*dt
th_deg = np.rad2deg(th)

thd = np.array([th_deg[0]+90, th_deg[1], th_deg[2]+90, th_deg[3], -th_deg[4]+90])